In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [5]:
df=pd.read_csv("/kaggle/input/remote-work-and-mental-health/Impact_of_Remote_Work_on_Mental_Health.csv")
df=df.fillna("nil")

In [6]:
df.columns

Index(['Employee_ID', 'Age', 'Gender', 'Job_Role', 'Industry',
       'Years_of_Experience', 'Work_Location', 'Hours_Worked_Per_Week',
       'Number_of_Virtual_Meetings', 'Work_Life_Balance_Rating',
       'Stress_Level', 'Mental_Health_Condition',
       'Access_to_Mental_Health_Resources', 'Productivity_Change',
       'Social_Isolation_Rating', 'Satisfaction_with_Remote_Work',
       'Company_Support_for_Remote_Work', 'Physical_Activity', 'Sleep_Quality',
       'Region'],
      dtype='object')

In [7]:
df.sample()

,Employee_ID,Age,Gender,Job_Role,Industry,Years_of_Experience,Work_Location,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Stress_Level,Mental_Health_Condition,Access_to_Mental_Health_Resources,Productivity_Change,Social_Isolation_Rating,Satisfaction_with_Remote_Work,Company_Support_for_Remote_Work,Physical_Activity,Sleep_Quality,Region
3573,EMP3574,22,Female,Project Manager,IT,35,Remote,50,0,5,Medium,Depression,Yes,Increase,2,Neutral,3,Daily,Poor,North America


In [8]:
df.isnull().sum()

Employee_ID                          0
Age                                  0
Gender                               0
Job_Role                             0
Industry                             0
Years_of_Experience                  0
Work_Location                        0
Hours_Worked_Per_Week                0
Number_of_Virtual_Meetings           0
Work_Life_Balance_Rating             0
Stress_Level                         0
Mental_Health_Condition              0
Access_to_Mental_Health_Resources    0
Productivity_Change                  0
Social_Isolation_Rating              0
Satisfaction_with_Remote_Work        0
Company_Support_for_Remote_Work      0
Physical_Activity                    0
Sleep_Quality                        0
Region                               0
dtype: int64

In [11]:
# one_hot=OneHotEncoder(sparse_output=False)
# one=one_hot.fit_transform(df[["Gender","Job_Role","Industry","Work_Location","Stress_Level","Mental_Health_Condition","Access_to_Mental_Health_Resources",""]])

In [12]:
new_df=pd.get_dummies(df).astype(int)

In [13]:
new_df

,Age,Years_of_Experience,Hours_Worked_Per_Week,Number_of_Virtual_Meetings,Work_Life_Balance_Rating,Social_Isolation_Rating,Company_Support_for_Remote_Work,Employee_ID_EMP0001,Employee_ID_EMP0002,Employee_ID_EMP0003,Employee_ID_EMP0004,Employee_ID_EMP0005,Employee_ID_EMP0006,Employee_ID_EMP0007,Employee_ID_EMP0008,Employee_ID_EMP0009,Employee_ID_EMP0010,Employee_ID_EMP0011,Employee_ID_EMP0012,Employee_ID_EMP0013,Employee_ID_EMP0014,Employee_ID_EMP0015,Employee_ID_EMP0016,Employee_ID_EMP0017,Employee_ID_EMP0018,Employee_ID_EMP0019,Employee_ID_EMP0020,Employee_ID_EMP0021,Employee_ID_EMP0022,Employee_ID_EMP0023,Employee_ID_EMP0024,Employee_ID_EMP0025,Employee_ID_EMP0026,Employee_ID_EMP0027,Employee_ID_EMP0028,Employee_ID_EMP0029,Employee_ID_EMP0030,Employee_ID_EMP0031,Employee_ID_EMP0032,Employee_ID_EMP0033,...,Job_Role_Project Manager,Job_Role_Sales,Job_Role_Software Engineer,Industry_Consulting,Industry_Education,Industry_Finance,Industry_Healthcare,Industry_IT,Industry_Manufacturing,Industry_Retail,Work_Location_Hybrid,Work_Location_Onsite,Work_Location_Remote,Stress_Level_High,Stress_Level_Low,Stress_Level_Medium,Mental_Health_Condition_Anxiety,Mental_Health_Condition_Burnout,Mental_Health_Condition_Depression,Mental_Health_Condition_nil,Access_to_Mental_Health_Resources_No,Access_to_Mental_Health_Resources_Yes,Productivity_Change_Decrease,Productivity_Change_Increase,Productivity_Change_No Change,Satisfaction_with_Remote_Work_Neutral,Satisfaction_with_Remote_Work_Satisfied,Satisfaction_with_Remote_Work_Unsatisfied,Physical_Activity_Daily,Physical_Activity_Weekly,Physical_Activity_nil,Sleep_Quality_Average,Sleep_Quality_Good,Sleep_Quality_Poor,Region_Africa,Region_Asia,Region_Europe,Region_North America,Region_Oceania,Region_South America
0,32,13,47,7,2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0
1,40,3,52,4,1,3,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0
2,59,22,46,11,5,4,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0
3,27,20,32,8,4,3,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
4,49,32,35,12,2,3,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,32,4,24,2,5,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0
4996,39,27,48,15,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0
4997,42,21,34,1,4,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0
4998,27,26,58,0,5,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0


# # AutoEncoders

In [32]:
from keras.layers import Dense,LSTM,SimpleRNN,GRU,BatchNormalization,Dropout,Input,TimeDistributed
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [25]:
early_stopping=EarlyStopping(
monitor="val_loss",
patience=5,
verbose=1)

In [26]:
new_df2=new_df.values
new_df2=np.repeat(new_df2[:,np.newaxis,:],10,axis=1)

In [27]:
new_df2.shape

(5000, 10, 5055)

In [38]:
inputs=Input(shape=(10,5055))
S=SimpleRNN(32,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)
S=LSTM(64,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)
S=GRU(128,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=GRU(256,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)

S=SimpleRNN(128,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)
S=LSTM(64,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)
S=GRU(32,return_sequences=True,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(S)
S=Dropout(0.5)(S)
S=BatchNormalization()(S)
outputs=TimeDistributed(Dense(5055,activation="softmax"))(S)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 10, 5055)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_11 (SimpleRNN)       │ (None, 10, 32)         │       162,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_35 (Dropout)            │ (None, 10, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 10, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 10, 64)         │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_36 (Dropout)            │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 10, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_15 (GRU)                    │ (None, 10, 128)        │        74,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_37 (Dropout)            │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 10, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_16 (GRU)                    │ (None, 10, 256)        │       296,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 10, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 10, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_12 (SimpleRNN)       │ (None, 10, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 10, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 10, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_11 (LSTM)                  │ (None, 10, 64)         │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 10, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_17 (GRU)                    │ (None, 10, 32)         │         9,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 10, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 10, 32)         │           12

 Total params: 836,319 (3.19 MB)

 Trainable params: 834,911 (3.18 MB)

 Non-trainable params: 1,408 (5.50 KB)

In [39]:
model.fit(new_df2,new_df2,epochs=32,batch_size=32,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/32
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 121ms/step - accuracy: 0.0963 - loss: 1092.6230 - val_accuracy: 0.0095 - val_loss: 1640.6277
Epoch 2/32
125/125 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.3913 - loss: 688.1151 - val_accuracy: 0.2470 - val_loss: 1213.7122
Epoch 3/32
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 102ms/step - accuracy: 0.5058 - loss: 385.5323 - val_accuracy: 0.5280 - val_loss: 296.7967
Epoch 4/32
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 106ms/step - accuracy: 0.5168 - loss: 320.6758 - val_accuracy: 0.5280 - val_loss: 295.1471
Epoch 5/32
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 105ms/step - accuracy: 0.5250 - loss: 304.5662 - val_accuracy: 0.5280 - val_loss: 294.4800
Epoch 6/32
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 106ms/step - accuracy: 0.5328 - loss: 297.2121 - val_accuracy: 0.5280 - val_loss: 293.9046
Epoch 7/32
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 103ms/step - accuracy: 0.5294 - loss: 296.1606 - val_accuracy: 0.5280 - val_loss: 293.4388
Epoch 8/32
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 103ms/st

In [40]:
prediction=model.predict(new_df2)
reconstruction_error_normal=np.mean(np.square(new_df2-prediction))
threshold=np.mean(reconstruction_error_normal)+2*np.std(reconstruction_error_normal)
print(reconstruction_error_normal,threshold)

157/157 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step
0.7919804172131628 0.7919804172131628


In [49]:
from sklearn.metrics import *
test_data=new_df2[1234:1345,:]
prediction_test=model.predict(test_data)

reconstruction_error_test=np.mean(np.square(test_data-prediction_test))

label=np.where(reconstruction_error_test>threshold,0,1)
accuracy=np.mean(label==0)*100
print(reconstruction_error_test,label,accuracy,np.argmax(prediction_test))
predictions_binary = (prediction_test > 0.5).astype(int)
predictions_binary=predictions_binary.flatten()
test_data=test_data.flatten()
print(accuracy_score(test_data,predictions_binary))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
0.7994041336829418 0 100.0 2269695
0.9960559966494684
